In [113]:
from Graph import Graph

# initialize graph
graph = Graph()

In [114]:
import pandas as pd

# helper function to add a group of connected players to graph
# typical usage: sending an array of players on the same team
def addPlayersToGraph(graph, players):
    numPlayers = len(players)

    # iterate through each player
    for i in range(0, numPlayers):
        player0 = players[i]

        # add connections to all players that aren't the 'player0'
        for j in range(i + 1, numPlayers):
            player1 = players[j]
            graph.addNeighbor(player0, player1)

players = {}

# read all csv files containing players and place into a graph
end_year = 2021
for year in range(2020, end_year + 1):
    df = pd.read_csv(f"player_data/nfl_players_data_{year}.csv")

    teams = {}

    # add each player to team
    for row in df.iterrows():
        content = row[1]
        team = content["team"]

        if teams.get(team) == None:
            teams[team] = []
        
        teams[team].append(content["playerid"])
        players[content["playerid"]] = {
            "team": team[:-4],
            "name": content["player_name"]
        }

    # fill in teams
    for team in teams:
        addPlayersToGraph(graph, teams[team])

In [117]:
import networkx as nx 
networkG = nx.Graph(graph. connections)
pos = nx.fruchterman_reingold_layout(networkG)

import math

output = {
    "edges": [],
    "nodes": []
}

team_colors = {
    "buf": "#00338D",
    "mia": "#008E97",
    "nyj": "#203731",
    "nwe": "#0C2340",
    "oti": "#0C2340",
    "clt": "#002C5F",
    "jax": "#006778",
    "htx": "#03202F",
    "rav": "#241773",
    "cin": "#FB4F14",
    "cle": "#311D00",
    "pit": "#FFB612",
    "kan": "#E31837",
    "sdg": "#0072C6",
    "den": "#FB4F14",
    "rai": "#000000",
    "phi": "#004C54",
    "dal": "#041E42",
    "nyg": "#0B2265",
    "was": "#773141",
    "tam": "#D50A0A",
    "atl": "#A71930",
    "nor": "#D3BC8D",
    "car": "#0085CA",
    "min": "#4F2683",
    "gnb": "#203731",
    "det": "#0076B6",
    "chi": "#0B162A",
    "sfo": "#AA0000",
    "crd": "#97233F",
    "ram": "#002244",
    "sea": "#002244"
}

max_edges = 0

# fill edges
for source, targets in graph.connections.items():
    max_edges = max(max_edges, len(targets))
    for target in targets:
        output["edges"].append({
            "source": source,
            "target": target,
            "attributes": {
                "color": team_colors[players[source]["team"]] + "32"
            }
        })


largest_node_size = 7.5


# fill nodess
for key, value in pos.items():
    output["nodes"].append({
        "key": key,
        "attributes": {
            "x": value[0],
            "size": len(graph.connections[key])/max_edges * largest_node_size,
            "color": team_colors[players[key]["team"]],
            "label": players[key]["name"],
            "y": value[1]
        }
    })

import json

with open("data.json", "w") as outfile:
    json.dump(output, outfile)
